# Advanced: Switches & Skeletons

In [4]:
from robotic import ry
import numpy as np
import time

In [7]:
C = ry.Config()

C.addFile("model2.g")
C.view()

0

In [9]:
komo = ry.KOMO()

komo = ry.KOMO(C, 2.5, 30, 2, False)
# komo.setTiming(2.5, 30, 5., 2)

In [10]:
# komo.add_qControlObjective([], 2)
komo.addControlObjective([], 2, 1e0)

In [11]:
komo.addQuaternionNorms([], 3.)

In [14]:
#grasp
komo.addModeSwitch([1., 2.], ry.SY.stable, ["gripper", "box"], True)
komo.addObjective([1.], ry.FS.positionDiff, ["gripper", "box"], ry.OT.eq, [1e2])
komo.addObjective([1.], ry.FS.scalarProductXX, ["gripper", "box"], ry.OT.eq, [1e2], [0.])
komo.addObjective([1.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

#slow - down - up
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([.9,1.1], ry.FS.position, ["gripper"], ry.OT.eq, [], [0.,0.,.1], 2)

#place
komo.addModeSwitch([2., -1.], ry.SY.stable, ["table", "box"])
komo.addObjective([2.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([2.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

#slow - down - up
komo.addObjective([2.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([1.9,2.2], ry.FS.position, ["gripper"], ry.OT.eq, [], [0.,0.,.1], 2)

In [15]:
# #komo.verbose = 4;
# komo.optimize();
# #/  komo.checkGradients();

# komo.view(False, "result")
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.118565, evals: 48, done: 1, feasible: 0, sos: 106.533, f: 0, ineq: 0, eq: 11.9607 }
size of path: (75, 7)


In [16]:
komo.view_play(False, .1)

0

## Same again, but only the keyframes:

In [205]:
komo=0

In [10]:
komo = ry.KOMO()
komo.setConfig(C, False)
komo.setTiming(3., 1, 5., 1) #DIFFERENT

In [11]:
komo.addControlObjective([], 1, 1e-1) #DIFFERENT
komo.addQuaternionNorms([], 3.)

In [12]:
#grasp
komo.addModeSwitch([1., 2.], ry.SY.stable, ["gripper", "box"])
komo.addObjective([1.], ry.FS.positionDiff, ["gripper", "box"], ry.OT.eq, [1e2])
komo.addObjective([1.], ry.FS.scalarProductXX, ["gripper", "box"], ry.OT.eq, [1e2], [0.])
komo.addObjective([1.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

#DIFFERENT: UP-DOWN MISSING
    
#place
komo.addModeSwitch([2., -1.], ry.SY.stable, ["table", "box"])
komo.addObjective([2.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([2.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

In [13]:
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

komo.view(False, "result")
# print("constraints:", komo.getConstraintViolations())
# print("costs:", komo.getCosts())
# komo.getReport()

{ time: 0.003061, evals: 14, done: 1, feasible: 0, sos: 0.00787156, f: 0, ineq: 0, eq: 8.63145 }
size of path: (3, 7)


0

In [14]:
komo.view_play(True, .5)

1

## Skeletons

In [1]:
import sys
sys.path += ['../../lib', '../../build', '../../../build']
import numpy as np
from robotic import ry

C = ry.Config()
C.addFile("model.g")
C.view()

0

In [2]:
S = ry.Skeleton()

In [3]:
# S.addEntry([1.], ry.SY.topBoxGrasp, ["gripper", "box2"])
S.addEntry([1., 1.], ry.SY.touch,  ["gripper", "box2"])
S.addEntry([1., 2.], ry.SY.stable, ["gripper", "box2"])
S.addEntry([0.8, 1.2], ry.SY.downUp, ["gripper"])
S.addEntry([1.8, 2.2], ry.SY.downUp, ["gripper"])
S.addEntry([2., 2.], ry.SY.poseEq, ["box2", "target2"])
S.addEntry([2., -1], ry.SY.stable, ["table", "box2"])
# S.enableAccumulatedCollisions(True)

In [13]:
#komo = S.getKomo_waypoints(C, 0.1, 0.01, 0.0)

lenScale, homingScale, and collScale are parameters that are likely used for some kind of optimization or cost function. Here's a brief explanation of each:

    lenScale (Length Scale): This parameter is likely used to control the relative importance of the length or distance-related terms in the optimization. A higher value would mean that length-related terms have more influence on the optimization, while a lower value would reduce their impact.

    homingScale (Homing Scale): This parameter might be related to a "homing" behavior, which typically means returning to a specific position or configuration. It could control how strongly the algorithm prioritizes returning to a specific home position.

    collScale (Collision Scale): This parameter likely controls how much the optimization considers avoiding collisions. A higher value would mean that avoiding collisions is more important, while a lower value would reduce its importance.

In [16]:
## solve for waypoints: create a komo instance, create nlp instance, then call generic solver

komo = S.getKomo_path(C, stepsPerPhase=30, accScale=1e0, lenScale=1e-2, homingScale=1e-1, collScale=1e0);

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

komo.view(False, "result")


{ time: 0.041698, evals: 25, done: 1, feasible: 1, sos: 1.28658, f: 0, ineq: 0, eq: 0.0277194 }
size of path: (66, 13)


0

### Alternative Method

In [4]:
komo = S.getKomo_path(C, stepsPerPhase=30, accScale=1e0, lenScale=1e-2, homingScale=1e-1, collScale=1e0);
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
# report on result, view, and play
print(ret)

{ time: 0.048465, evals: 22, done: 1, feasible: 1, sos: 1.27302, f: 0, ineq: 0, eq: 0.0300044 }====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1

==nlp== it:   0  evals:  12  A(x):    1.12189  f:   0.985722  g:          0  h:    2.54408  |x-x'|:     1.3937 	stop:DeltaConverge
==nlp== it:   1  evals:  12  A(x):    1.93893  mu:5
==nlp== it:   1  evals:  18  A(x):     1.2531  f:    1.20582  g:          0  h:    0.49554  |x-x'|:   0.165301 	stop:DeltaConverge
==nlp== it:   2  evals:  18  A(x):    1.36325  mu:25
==nlp== it:   2  evals:  21  A(x):    1.27986  f:    1.25835  g:          0  h:   0.127686  |x-x'|:  0.0128698 	stop:DeltaConverge
==nlp== it:   3  evals:  21  A(x):    1.32631  mu:125
==nlp== it:   3  evals:  22  A(x):    1.27562  f:    1.27302  g:          0  h:  0.0300044  |x-x'|: 0.00802102 	stop:DeltaConverge
==nlp== StoppingCriterion Delta<0.01


In [5]:
#komo.view(True, 'waypoints solution')
komo.view_play(True, .2)

1

## Skeletons_2

In [2]:
import sys
sys.path.append('../build')
import time
import numpy as np
from robotic import ry
#import libry as ry

## create a configuration
C = ry.Config()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/tables.g')
C.addFrame('obj0', 'table1', 'type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:"t(0 0 .15)"' )
C.addFrame('obj1', 'table1', 'type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:"t(0 .2 .15)"' )
C.addFrame('obj2', 'table1', 'type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:"t(0 .4 .15)"' )
C.addFrame('obj3', 'table1', 'type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:"t(0 .6.15)"' )
C.addFrame('tray', 'table2', 'type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:"t(0 0 .07)"' );
C.addFrame('', 'tray', 'type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]' )
C.view(False, 'initial model')


0

In [3]:

## create a skeleton
S = ry.Skeleton()
S.addEntry([1.], ry.SY.touch, ['pr2R', 'obj0'])
S.addEntry([1., 3.], ry.SY.stable, ['pr2R', 'obj0'])
S.addEntry([2.,2.], ry.SY.touch, ['pr2L', 'obj3'])
S.addEntry([2.,4.], ry.SY.stable, ['pr2L', 'obj3'])
S.addEntry([3.], ry.SY.above, ['obj0', 'tray'])
S.addEntry([3.,4.], ry.SY.stableOn, ['tray', 'obj0'])

## solve for waypoints: create a komo instance, create nlp instance, then call generic solver
komo = S.getKomo_waypoints(C, 1e-1, 1e-2,0)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
# report on result, view, and play
print(ret)
#print(nlpW.report(2))
komo.view(True, 'waypoints solution')
komo.view_play(True, .2)
# store result

{ time: 0.127526, evals: 305, done: 1, feasible: 1, sos: 0.434056, f: 0, ineq: 0, eq: 0.00561112 }
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals: 100  A(x):    0.48276  f:   0.480654  g:  0.0242739  h:  0.0946715  |x-x'|:    1.71951 	stop:DeltaConverge
==nlp== it:   1  evals: 100  A(x):   0.495395  mu:5
==nlp== it:   1  evals: 303  A(x):   0.433916  f:    0.43416  g:          0  h: 0.00977696  |x-x'|:   0.615392 	stop:DeltaConverge
==nlp== it:   2  evals: 303  A(x):   0.434463  mu:25
==nlp== it:   2  evals: 305  A(x):   0.434152  f:   0.434056  g:          0  h: 0.00561112  |x-x'|: 0.00189657 	stop:DeltaConverge
==nlp== StoppingCriterion Delta<0.01


1

In [4]:
## solve for paths using RRT: for each phase create start-end problems, run RRT
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for t in range(0,int(m)):
    # grab config and waypoints
    [Ctmp, q0, q1] = S.getTwoWaypointProblem(t, komo)
    Ctmp.setJointState(q0);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' START')
    Ctmp.setJointState(q1);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' STOP')

    Ctmp.view(True, 'Continue')

    # call path finder
    sol = ry.PathFinder()
    sol.setProblem(Ctmp, q0, q1)
    ret = sol.solve()
    rrt_paths.append(ret.x)
    rrt_dofs.append(Ctmp.getDofIDs())

    #display the rrt path
    for i in range(0,ret.x.shape[0]):
        Ctmp.setJointState(ret.x[i])
        Ctmp.view(False, 'rrt path ' + str(i))
        time.sleep(.02)

In [12]:
## solve for full path: create a komo instance, initialize with waypoints & rrt paths, solve
#komo = S.getKomo_path(C, 20, .2, -1, 1e-2)
#(self: robotic.ry.Skeleton, Configuration, stepsPerPhase: int, accScale: float, lenScale: float, homingScale: float, collScale: float)
komo = S.getKomo_path(C, 60, 1e0, 1e0, 1e0,1e0)
komo.initWithWaypoints(waypoints)
komo.view(True, 'init with waypoints only')
for t in range(0,int(m)):
    komo.initPhaseWithDofsPath(t, rrt_dofs[t], rrt_paths[t], True)
    komo.view(True, 'init with RRT phase ' + str(t))

nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
# report on result, view, and play
print(ret)
#print(nlp.report(2))
komo.view(True, 'path solution')
komo.view_play(True, .2)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
{ time: 7.30955, evals: 1000, done: 1, feasible: 0, sos: 9887.07, f: 0, ineq: 0.4, eq: 35.8865 }
==nlp== it:   0  evals: 135  A(x):    10298.2  f:    8906.61  g:        0.4  h:    136.599  |x-x'|:    4.22926 	stop:LineSearchSteps (bad:1)
==nlp== it:   1  evals: 135  A(x):    18650.1  mu:5
==nlp== it:   1  evals: 184  A(x):    16482.8  f:    9273.71  g:        0.4  h:    105.286  |x-x'|:    0.10313 	stop:TinyXSteps (bad:2)
==nlp== it:   2  evals: 184  A(x):    46187.9  mu:25
==nlp== it:   2  evals:1000  A(x):      22971  f:    9887.07  g:        0.4  h:    35.8865  |x-x'|:   0.280541 	stop:CritEvals (bad:3)
==nlp== StoppingCriterion MAX EVALS


1